In [1]:
import os
os.chdir("../")

In [2]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataPreprocessConfig:
    root_dir: Path
    load_file_path: Path
    local_file: Path
    params_select_columns: list

In [3]:
from PhishingDomainDetection.constants import CONFIG_FILE_PATH, PARAMS_FILE_APTH
from PhishingDomainDetection.utils import read_yaml_file, create_directories
from PhishingDomainDetection.entity import DataIngestionConfig

In [145]:
DataIngestionConfig

PhishingDomainDetection.entity.config_entity.DataIngestionConfig

In [146]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_APTH):
        self.config = read_yaml_file(config_file_path)
        self.params = read_yaml_file(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_data_load_for_preprocess(self):
        self.data_load_config = self.config.data_ingestion
        self.root_dir = self.data_load_config.root_dir
        self.file_dir = self.data_load_config.local_file
        self.file_path = os.path.join(self.root_dir, self.file_dir)
        return self.file_path
       

    def get_data_preprocess_config(self) -> DataPreprocessConfig:
        data_preprocessing = self.config.data_preprocessing
        dataset_load = os.path.join(self.config.data_ingestion.root_dir, "data.csv") 
        


        create_directories([data_preprocessing.root_dir])

        data_preprocess_config = DataPreprocessConfig(
            root_dir=Path(data_preprocessing.root_dir),
            load_file_path=Path(dataset_load),
            local_file=Path(data_preprocessing.local_file),
            params_select_columns=self.params.select_columns
        )

        return data_preprocess_config

In [147]:
class AA:
    def hello(self):
        self.hii = "kkk"
        print(self.hii)
    def hello2(self):
        self.hello()

In [148]:
cc = AA()
cc.hello2()

kkk


In [161]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split      
from sklearn.preprocessing import StandardScaler  

In [165]:
class DataPreprocess:
    def __init__(self, config: DataPreprocessConfig):
        self.config = config

    def data_load(self):
        data_path = self.config.load_file_path
        data = pd.read_csv(data_path)
        select_columns = self.config.params_select_columns
        self.df = data[select_columns]
    
    def handling_missing_value(self):
        # we have already done research on missing values. you can refer research/preprocessing.ipynb
        # delete params_length because it's having 91% missing values
        self.df = self.df.drop('params_length', axis=1)

        # fill missing values with median
        self.df = self.df.replace(-1, np.nan)
        self.df = self.df.fillna(self.df.median())

    def handling_imbalanced_data(self):
        sm = SMOTE(random_state = 33)
        self.df_new, df_phishing_new = sm.fit_resample(self.df.drop('phishing', axis=1), self.df.phishing)
        self.df_new['phishing'] = df_phishing_new

        
    def sacling_data(self):
        scaler = StandardScaler()
        self.df_scaled = pd.DataFrame(scaler.fit_transform(self.df_new.drop('phishing', axis=1)),columns = self.df.columns[:-1])
        self.df_scaled['phishing'] = self.df_new.phishing

    def saving_preprocessing_data(self):
        data_preprocessing_root_dir = self.config.root_dir
        local_file_dir = self.config.local_file

        raw_local_dir_path = os.path.join(data_preprocessing_root_dir, local_file_dir)

        self.df_scaled.to_csv(raw_local_dir_path, index=False)


In [166]:
config = ConfigurationManager()
data_preprocess_config = config.get_data_preprocess_config()
data_preprocess = DataPreprocess(config=data_preprocess_config)
data_preprocess.data_load()
data_preprocess.handling_missing_value()
data_preprocess.handling_imbalanced_data()
data_preprocess.sacling_data()
data_preprocess.saving_preprocessing_data()



[2023-02-04 20:17:48,691: INFO common]: yaml file configs\config.yaml load  successfully
[2023-02-04 20:17:48,699: INFO common]: yaml file params.yaml load  successfully
[2023-02-04 20:17:48,701: INFO common]: created directory at artifacts
[2023-02-04 20:17:48,703: INFO common]: created directory at artifacts/data_preprocessing
